# Quantum Computing Moons Example 

9.1. Follow the tutorial at the `https://pennylane.ai/qml/demos/tutorial_qnn_module_tf.html` and create a quantum neural network to classify the Moons Dataset!

9.2. Compare the quantum neural network to a classic neural network!

In [ ]:
!conda install -q scikit-learn
!pip install -q pennylane
!pip install -q tensorflow
!pip install -q matplotlib

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons

In [ ]:
# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

X, y = make_moons(n_samples=200, noise=0.1)
y_hot = tf.keras.utils.to_categorical(y, num_classes=2)  # one-hot encoded labels

c = ["#1f77b4" if y_ == 0 else "#ff7f0e" for y_ in y]  # colours for each class
plt.axis("off")
plt.scatter(X[:, 0], X[:, 1], c=c)
plt.show()

In [ ]:
import pennylane as qml

n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
clayer_1 = tf.keras.layers.Dense(2)
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")

# Model 1 without quantum

In [ ]:
model1 = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model1.compile(opt, loss="mae", metrics=["accuracy"])

In [ ]:
fitting = model1.fit(X, y_hot, epochs=10, batch_size=5, validation_split=0.25,verbose=2)

# Model 2 with Quantum

In [ ]:
model2 = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model2.compile(opt, loss="mae", metrics=["accuracy"])

In [ ]:
fitting = model2.fit(X, y_hot, epochs=10, batch_size=5, validation_split=0.25,verbose=2)
